In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
import sys
from logging import DEBUG, INFO, WARNING, StreamHandler, getLogger

logger = getLogger()
if not logger.hasHandlers():
    logger.addHandler(StreamHandler(sys.stdout))
logger.setLevel(INFO)

# Import libraries

In [ ]:
import gc
import glob
import os
import pathlib
import time
import typing

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
import torch.nn.functional as F
from cfd_model.cfd.periodic_channel_domain import TorchSpectralModel2D
from cfd_model.enkf.sr_enkf import (
    calc_localization_matrix,
    hr_assimilate_with_existing_data,
)
from cfd_model.filter.low_pass_periodic_channel_domain import LowPassFilter
from cfd_model.initialization.periodic_channel_jet_initializer import (
    calc_init_omega,
    calc_init_perturbation_hr_omegas,
    calc_jet_forcing,
)
from cfd_model.interpolator.torch_interpolator import (
    interpolate,
    interpolate_time_series,
)
from src.dataloader import split_file_paths
from src.dataset import generate_is_obs_and_obs_matrix
from src.ssim import SSIM
from src.utils import read_pickle, set_seeds, write_pickle
from torch.distributions.multivariate_normal import MultivariateNormal
from tqdm.notebook import tqdm

In [ ]:
os.environ["CUBLAS_WORKSPACE_CONFIG"] = r":4096:8"  # to make calculations deterministic
set_seeds(42, use_deterministic=True)

# Define constants

In [ ]:
ROOT_DIR = str((pathlib.Path(os.environ["PYTHONPATH"]) / "..").resolve())
ROOT_DIR

In [ ]:
DEVICE = "cuda:2"
if not torch.cuda.is_available():
    raise Exception("No GPU. CPU is used.")

In [ ]:
ENKF_PREFERENCES = {
    4: {
        "INIT_SYS_NOISE_FACTOR": 0.0,
        "LOCALIZE_DX": 0.15,
        "N_ENS": 100,
        "OBS_PERTURB_STD": 1e-05,
        "SYS_NOISE_FACTOR": 0.0005,
    },
    6: {
        "INIT_SYS_NOISE_FACTOR": 0.2,
        "LOCALIZE_DX": 0.3,
        "N_ENS": 100,
        "OBS_PERTURB_STD": 0.03,
        "SYS_NOISE_FACTOR": 0.002,
    },
    8: {
        "INIT_SYS_NOISE_FACTOR": 0.2,
        "LOCALIZE_DX": 0.5,
        "N_ENS": 100,
        "OBS_PERTURB_STD": 0.10,
        "SYS_NOISE_FACTOR": 0.005,
    },
    10: {
        "INIT_SYS_NOISE_FACTOR": 0.2,
        "LOCALIZE_DX": 0.7,
        "N_ENS": 100,
        "OBS_PERTURB_STD": 0.20,
        "SYS_NOISE_FACTOR": 0.005,
    },
    12: {
        "INIT_SYS_NOISE_FACTOR": 0.2,
        "LOCALIZE_DX": 0.7,
        "N_ENS": 100,
        "OBS_PERTURB_STD": 0.20,
        "SYS_NOISE_FACTOR": 0.005,
    },
}

In [ ]:
GRID_INTERVAL = 8
I_SEED_UHR = 9996
OBS_SRDA_SEED = 771155
# 221958, 771155, 832180, 465838, 359178

In [ ]:
CONFIG_DIR = f"{ROOT_DIR}/pytorch/config/paper_experiment_06"
CONFIG_NAME = f"lt4og{GRID_INTERVAL:02}_on1e-01_ep1000_lr1e-04_scT_bT_muT_a02_b02_sd{OBS_SRDA_SEED}"
CONFIG_PATH = f"{CONFIG_DIR}/{CONFIG_NAME}.yml"

In [ ]:
TMP_DATA_DIR = "./data"
CSV_DATA_DIR = "./csv"
FIG_DIR = "./tmp/fig"

In [ ]:
UHR_RESULT_DIR = f"{ROOT_DIR}/data/pytorch/CFD/jet27/seed{I_SEED_UHR:05}"

CFD_DIR_NAME = "jet12"
TRAIN_VALID_TEST_RATIOS = [0.7, 0.2, 0.1]

INFLATION = 1.0
SEED = 98765

ASSIMILATION_PERIOD = 4
START_TIME_INDEX = 0
MAX_TIME_INDEX_FOR_INTEGRATION = 96

NUM_TIMES = MAX_TIME_INDEX_FOR_INTEGRATION

LR_NX = 32
LR_NY = 17
LR_DT = 5e-4
LR_NT = 500

HR_NX = 128
HR_NY = 65
HR_DT = LR_DT / 4.0
HR_NT = LR_NT * 4

UHR_NX = 1024
UHR_NY = 513

Y0 = np.pi / 2.0
SIGMA = 0.4
U0 = 3.0
TAU0 = 0.3
PERTUB_NOISE = 0.0025

BETA = 0.1
COEFF_LINEAR_DRAG = 1e-2
ORDER_DIFFUSION = 2
HR_COEFF_DIFFUSION = 1e-5

DT = LR_DT * LR_NT
INIT_TIME_INDEX = START_TIME_INDEX

HR_CFD_CONFIG = {
    "nx": HR_NX,
    "ny": HR_NY,
    "hr_nx": HR_NX,
    "hr_ny": HR_NY,
    "assimilation_period": ASSIMILATION_PERIOD,
    "coeff_linear_drag": COEFF_LINEAR_DRAG,
    "coeff_diffusion": HR_COEFF_DIFFUSION,
    "order_diffusion": ORDER_DIFFUSION,
    "beta": BETA,
    "device": DEVICE,
    "y0": Y0,
    "sigma": SIGMA,
    "tau0": TAU0,
    "t0": 0.0,
}

In [ ]:
OBS_GRID_RATIO = {
    0: 0.0,
    4: 0.06250000093132257,
    5: 0.03999999910593033,
    6: 0.027777777363856632,
    7: 0.02040816326530612,
    8: 0.015625000116415322,
    9: 0.012345679127323775,
    10: 0.010000000149011612,
    11: 0.008264463206306716,
    12: 0.006944444625534945,
    13: 0.005917159876284691,
    14: 0.005102040977882487,
    15: 0.004444444572759999,
    16: 0.003906250014551915,
}

In [ ]:
N_ENS = int(ENKF_PREFERENCES[GRID_INTERVAL]["N_ENS"])
LOCALIZE_DX = ENKF_PREFERENCES[GRID_INTERVAL]["LOCALIZE_DX"]
SYS_NOISE_FACTOR = ENKF_PREFERENCES[GRID_INTERVAL]["SYS_NOISE_FACTOR"]
INIT_SYS_NOISE_FACTOR = ENKF_PREFERENCES[GRID_INTERVAL]["INIT_SYS_NOISE_FACTOR"]
OBS_PERTURB_STD = ENKF_PREFERENCES[GRID_INTERVAL]["OBS_PERTURB_STD"]

LOCALIZE_DY = LOCALIZE_DX
HR_CFD_CONFIG["ne"] = int(N_ENS)
HR_CFD_CONFIG["n_ens"] = int(N_ENS)

# Define methods

In [ ]:
def get_uhr_and_hr_omegas():
    all_uhr_omegas = []
    for path in sorted(glob.glob(f"{UHR_RESULT_DIR}/*.npy")):
        uhr = torch.from_numpy(np.load(path)).squeeze()
        assert uhr.shape == (UHR_NX, UHR_NY)
        all_uhr_omegas.append(uhr)
    # Stack along time dim
    all_uhr_omegas = torch.stack(all_uhr_omegas)[:NUM_TIMES]
    assert all_uhr_omegas.shape == (NUM_TIMES, UHR_NX, UHR_NY)

    tmp = all_uhr_omegas[:, None, :, 1:]
    _omegas = F.avg_pool2d(tmp, kernel_size=8).squeeze()

    all_hr_omegas = torch.zeros((NUM_TIMES, HR_NX, HR_NY), dtype=_omegas.dtype)
    all_hr_omegas[:, :, 1:] = _omegas

    return all_uhr_omegas, all_hr_omegas

In [ ]:
def get_initial_hr_omega(ne: int):
    hr_jet, _ = calc_jet_forcing(
        nx=HR_NX,
        ny=HR_NY,
        ne=ne,
        y0=Y0,
        sigma=SIGMA,
        tau0=TAU0,
    )

    hr_perturb = calc_init_perturbation_hr_omegas(
        nx=HR_NX, ny=HR_NY, ne=ne, noise_amp=PERTUB_NOISE, seed=SEED
    )

    hr_omega0 = calc_init_omega(
        perturb_omega=hr_perturb,
        jet=hr_jet,
        u0=U0,
    )

    return hr_omega0

In [ ]:
def create_initialized_hr_model(hr_omega0: torch.Tensor, t0: float = 0.0):
    assert hr_omega0.shape == (HR_NX, HR_NY)

    omega0 = torch.broadcast_to(hr_omega0, (N_ENS, HR_NX, HR_NY))

    hr_model = TorchSpectralModel2D(**HR_CFD_CONFIG)
    _, hr_forcing = calc_jet_forcing(**HR_CFD_CONFIG)

    hr_model.initialize(t0=t0, omega0=omega0, forcing=hr_forcing)
    hr_model.calc_grid_data()

    return hr_model


def get_obs_matrices(obs_grid_interval: int):
    obs_matrices = []

    for init_x in tqdm(range(obs_grid_interval)):
        for init_y in range(obs_grid_interval):
            _, obs_mat = generate_is_obs_and_obs_matrix(
                nx=HR_NX,
                ny=HR_NY,
                init_index_x=init_x,
                init_index_y=init_y,
                interval_x=obs_grid_interval,
                interval_y=obs_grid_interval,
                dtype=torch.float64,
            )
            obs_matrices.append(obs_mat)

    return obs_matrices


def load_hr_data(
    root_dir: str,
    cfd_dir_name: str,
    train_valid_test_ratios: typing.List[str],
    kind: str,
    num_hr_omega_sets: int,
    max_ens_index: int = 20,
) -> torch.Tensor:

    cfd_dir_path = f"{root_dir}/data/pytorch/CFD/{cfd_dir_name}"
    logger.info(f"CFD dir path = {cfd_dir_path}")

    data_dirs = sorted([p for p in glob.glob(f"{cfd_dir_path}/*") if os.path.isdir(p)])

    train_dirs, valid_dirs, test_dirs = split_file_paths(
        data_dirs, train_valid_test_ratios
    )

    if kind == "train":
        target_dirs = train_dirs
    elif kind == "valid":
        target_dirs = valid_dirs
    elif kind == "test":
        target_dirs = test_dirs
    else:
        raise Exception(f"{kind} is not supported.")

    logger.info(f"Kind = {kind}, Num of dirs = {len(target_dirs)}")

    all_hr_omegas = []
    for dir_path in sorted(target_dirs):
        for i in range(max_ens_index):

            hr_omegas = []
            for file_path in sorted(glob.glob(f"{dir_path}/*_hr_omega_{i:02}.npy")):
                data = np.load(file_path)

                # This is to avoid overlapping at the start/end point
                if len(hr_omegas) > 0:
                    data = data[1:]
                hr_omegas.append(data)

            # Concat along time axis
            all_hr_omegas.append(np.concatenate(hr_omegas, axis=0))

            if len(all_hr_omegas) == num_hr_omega_sets:
                # Concat along batch axis
                ret = np.stack(all_hr_omegas, axis=0)
                return torch.from_numpy(ret).to(torch.float64)

    ret = np.stack(all_hr_omegas, axis=0)
    return torch.from_numpy(ret).to(torch.float64)


def get_obs_matrix(obs: torch.Tensor) -> torch.Tensor:

    assert obs.shape == (HR_NX, HR_NY)
    is_obs = torch.where(torch.isnan(obs), torch.zeros_like(obs), torch.ones_like(obs))

    obs_indices = is_obs.reshape(-1)
    obs_indices = torch.where(obs_indices == 1.0)[0]

    num_obs = len(obs_indices)

    obs_matrix = torch.zeros(num_obs, HR_NX * HR_NY, dtype=torch.float64, device=DEVICE)

    for i, j in enumerate(obs_indices):
        obs_matrix[i, j] = 1.0

    p = 100 * torch.sum(obs_matrix).item() / (HR_NX * HR_NY)
    logger.debug(f"observatio prob = {p} [%]")

    return obs_matrix


def calc_errors(all_gt: torch.Tensor, all_enkf: torch.Tensor) -> pd.DataFrame:
    ssim_func = SSIM(size_average=False, use_gauss=True)
    df_errors = pd.DataFrame()

    for kind in ["LR", "HR"]:

        if kind == "HR":
            gt = all_gt
        else:
            gt = interpolate_time_series(all_gt, LR_NX, LR_NY, "bicubic")

        if kind == "LR":
            pred = all_enkf
        else:
            pred = interpolate_time_series(all_enkf, HR_NX, HR_NY, "bicubic")

        # batch, time, x, and y dims
        assert gt.ndim == 4
        assert gt.shape == pred.shape

        mae = torch.mean(torch.abs(gt - pred), dim=(-2, -1))  # mean over x and y
        nrms = torch.mean(torch.abs(gt), dim=(-2, -1))
        maer = torch.mean(mae / nrms, dim=0)  # mean over batch dim

        ssim = ssim_func(
            img1=gt.to(DEVICE),
            img2=pred.to(DEVICE),
        )
        ssim = torch.mean(ssim, dim=(0, -2, -1))  # mean over batch, x, and y
        ssim = 1.0 - ssim

        assert ssim.shape == maer.shape

        df_errors[f"{kind}_MAER"] = maer.numpy()
        df_errors[f"{kind}_SSIMLoss"] = ssim.cpu().numpy()

    return df_errors


def get_sys_noise_generator(num_hr_omega_sets: int = 250, eps: float = 1e-12):
    hr_omegas = load_hr_data(
        root_dir=ROOT_DIR,
        cfd_dir_name=CFD_DIR_NAME,
        train_valid_test_ratios=TRAIN_VALID_TEST_RATIOS,
        kind="train",
        num_hr_omega_sets=num_hr_omega_sets,
    )
    # dims = batch, time, x, and y
    logger.info(f"hr_omega shape = {hr_omegas.shape}")

    lr_omegas = interpolate_time_series(hr_omegas, LR_NX, LR_NY, "bicubic")
    lr_omegas = lr_omegas - torch.mean(lr_omegas, dim=0, keepdim=True)

    lr_omegas = lr_omegas.reshape(lr_omegas.shape[:2] + (-1,))
    # dims = batch, time, and space

    del hr_omegas
    gc.collect()

    # Inner product over batch dim
    all_covs = torch.mean(lr_omegas[..., None, :] * lr_omegas[..., None], dim=0)

    # Assure conv is symmetric.
    all_covs = (all_covs + all_covs.permute(0, 2, 1)) / 2.0

    # Assure positive definiteness
    all_covs = all_covs + torch.diag(
        torch.full(size=(all_covs.shape[-1],), fill_value=eps)
    )

    loc = torch.zeros(all_covs.shape[-1], dtype=torch.float64)
    return [MultivariateNormal(loc, cov) for cov in all_covs]


def get_cov_for_sys_noise_generator(num_hr_omega_sets: int = 50, eps: float = 1e-10):
    hr_omegas = load_hr_data(
        root_dir=ROOT_DIR,
        cfd_dir_name=CFD_DIR_NAME,
        train_valid_test_ratios=TRAIN_VALID_TEST_RATIOS,
        kind="train",
        num_hr_omega_sets=num_hr_omega_sets,
    )
    # dims = batch, time, x, and y

    hr_omegas = hr_omegas.reshape(hr_omegas.shape[:2] + (-1,))
    hr_omegas = hr_omegas[:, ::ASSIMILATION_PERIOD]

    # Inner product over batch dim
    all_covs = torch.mean(hr_omegas[..., None, :] * hr_omegas[..., None], dim=0)

    # Assure conv is symmetric.
    all_covs = (all_covs + all_covs.permute(0, 2, 1)) / 2.0

    # Assure positive definiteness
    all_covs = all_covs + torch.diag(
        torch.full(size=(all_covs.shape[-1],), fill_value=eps)
    )

    return all_covs


class SysNoiseGenerator:
    def __init__(self, max_cycle_index: int = 25):
        all_noise = []
        for cycle in range(max_cycle_index):
            all_noise.append(np.load(f"{TMP_DATA_DIR}/sys_noise_assim_{cycle:03}.npy"))

        self.all_noise = torch.from_numpy(np.stack(all_noise, axis=0)).to(torch.float64)

    def __call__(self, cycle_index: int, num_noise: int = N_ENS):
        indices = np.arange(self.all_noise.shape[1])
        indices = np.random.choice(indices, size=num_noise, replace=True)

        return self.all_noise[cycle_index, indices]

# Perform enkf

In [ ]:
output_hr_enkf_file_path = f"{TMP_DATA_DIR}/UHR_seed_{I_SEED_UHR:05}_og{GRID_INTERVAL:02}_SRDA_seed_{OBS_SRDA_SEED}_enkf_hr_mean.npy"

In [ ]:
if os.path.exists(output_hr_enkf_file_path):
    raise Exception()

## Initialize

In [ ]:
output_obs_npz_file_path = (
    f"{TMP_DATA_DIR}/UHR_seed_{I_SEED_UHR:05}_og{GRID_INTERVAL:02}_{CONFIG_NAME}.npz"
)
all_data = np.load(output_obs_npz_file_path)
hr_obs = torch.from_numpy(all_data["hr_obs"])
sr_forecast = torch.from_numpy(all_data["sr_forecast"])

In [ ]:
hr_obs.shape

In [ ]:
cov_file_path = f"{TMP_DATA_DIR}/sys_noise_covs.pickle"

if os.path.exists(cov_file_path):
    all_covs = read_pickle(cov_file_path)
else:
    all_covs = get_cov_for_sys_noise_generator()
    write_pickle(all_covs, cov_file_path)

In [ ]:
torch_rand_generator = torch.Generator().manual_seed(SEED)

loc = torch.zeros(all_covs.shape[-1], dtype=torch.float64)
sys_noise_generators = [MultivariateNormal(loc, cov) for cov in all_covs]
sys_noise_generators = sys_noise_generators[INIT_TIME_INDEX // ASSIMILATION_PERIOD :]

localization_matrix = calc_localization_matrix(
    nx=HR_NX, ny=HR_NY, d_x=LOCALIZE_DX, d_y=LOCALIZE_DY
).to(DEVICE)

In [ ]:
assert len(sys_noise_generators) == (NUM_TIMES // ASSIMILATION_PERIOD + 1)

In [ ]:
set_seeds(SEED, use_deterministic=True)
init_hr_omega = get_initial_hr_omega(ne=1).squeeze()

In [ ]:
init_hr_omega.shape

In [ ]:
hr_model = create_initialized_hr_model(init_hr_omega)

## Perform EnKF (HR)

In [ ]:
start_time = time.time()

hr_enkfs = []
cnt_assim = 0

for i_cycle in tqdm(range(NUM_TIMES)):

    # Data assimilation
    if i_cycle > 0 and i_cycle % ASSIMILATION_PERIOD == 0:
        obs = hr_obs[i_cycle].to(torch.float64)
        obs_matrix = get_obs_matrix(obs)

        # This is to avoid nan when observation operator acts.
        obs = torch.nan_to_num(obs, nan=1e10)

        # This method returns forecast conv
        hr_assimilate_with_existing_data(
            hr_omega=obs.to(DEVICE),
            hr_ens_model=hr_model,
            obs_matrix=obs_matrix,
            obs_noise_std=OBS_PERTURB_STD,
            inflation=INFLATION,
            rand_generator=torch_rand_generator,
            localization_matrix=localization_matrix,
        )

    # Store data "after" assimilation
    hr_enkfs.append(hr_model.omega.cpu().clone())

    # Add additive system noise
    if i_cycle == 0 or (INFLATION == 1.0 and i_cycle % ASSIMILATION_PERIOD == 0):
        noise = sys_noise_generators[cnt_assim].sample([N_ENS])
        noise = noise.reshape(N_ENS, HR_NX, HR_NY)
        noise = noise - torch.mean(noise, dim=0, keepdims=True)
        assert noise.shape == hr_model.omega.shape

        factor = INIT_SYS_NOISE_FACTOR if i_cycle == 0 else SYS_NOISE_FACTOR
        omega = hr_model.omega + factor * noise.to(DEVICE)

        hr_model.initialize(t0=hr_model.t, omega0=omega)
        hr_model.calc_grid_data()
        cnt_assim += 1

    hr_model.time_integrate(dt=HR_DT, nt=HR_NT, hide_progress_bar=True)
    hr_model.calc_grid_data()

# Stack along time dim
hr_enkfs = torch.stack(hr_enkfs, dim=1).to(torch.float32)

# Add an ensemble mean
hr_mean_enkf = torch.mean(hr_enkfs, dim=0).numpy()
assert hr_mean_enkf.shape == (NUM_TIMES, HR_NX, HR_NY)

np.save(output_hr_enkf_file_path, hr_mean_enkf)

end_time = time.time()
logger.info(f"Elapsed Time = {end_time - start_time} [sec]\n")

# Plot

In [ ]:
def plot(
    dict_data: dict,
    t: float,
    obs: np.ndarray,
    gt_label: str,
    figsize: list = [20, 2],
    write_out: bool = False,
    ttl_header: str = "",
    fig_file_name: str = "",
    vmin_omega: float = -9,
    vmax_omega: float = 9,
):

    xs = np.linspace(0, 2 * np.pi, num=UHR_NX, endpoint=False)
    ys = np.linspace(0, np.pi, num=UHR_NY, endpoint=True)
    uhr_x, uhr_y = np.meshgrid(xs, ys, indexing="ij")

    xs = np.linspace(0, 2 * np.pi, num=HR_NX, endpoint=False)
    ys = np.linspace(0, np.pi, num=HR_NY, endpoint=True)
    hr_x, hr_y = np.meshgrid(xs, ys, indexing="ij")

    xs = np.linspace(0, 2 * np.pi, num=LR_NX, endpoint=False)
    ys = np.linspace(0, np.pi, num=LR_NY, endpoint=True)
    lr_x, lr_y = np.meshgrid(xs, ys, indexing="ij")

    plt.rcParams["font.family"] = "serif"
    plt.rcParams["font.size"] = 18
    fig, axes = plt.subplots(
        1, len(dict_data), figsize=figsize, sharex=True, sharey=False
    )

    gt = None
    for ax, (label, data) in zip(axes, dict_data.items()):
        d = np.squeeze(data)

        if d.shape == (UHR_NX, UHR_NY):
            x, y = uhr_x, uhr_y
        elif d.shape == (HR_NX, HR_NY):
            x, y = hr_x, hr_y
        else:
            x, y = lr_x, lr_y

        if label == gt_label:
            gt = d
            ttl = "Ground Truth"
        else:
            _d = interpolate(
                torch.from_numpy(d[None, ...]),
                nx=gt.shape[0],
                ny=gt.shape[1],
                mode="bicubic",
            )
            _d = _d.squeeze().numpy()
            assert _d.shape == gt.shape
            maer = np.mean(np.abs(gt - _d)) / np.mean(np.abs(gt))
            ttl = label
            ttl = f"{label}\nMAER={maer:.2f}"

        cnts = ax.pcolormesh(
            x, y, d, cmap="twilight_shifted", vmin=vmin_omega, vmax=vmax_omega
        )
        ax.set_title(ttl)

        fig.colorbar(
            cnts,
            ax=ax,
            ticks=[vmin_omega, vmin_omega / 2, 0, vmax_omega / 2, vmax_omega],
            extend="both",
        )

        ax.set_xlim([0, 2 * np.pi])
        ax.set_ylim([0, np.pi])

        if label == gt_label:
            o = (
                interpolate(
                    torch.from_numpy(obs[None, ...]),
                    nx=gt.shape[0],
                    ny=gt.shape[1],
                    mode="nearest",
                )
                .squeeze()
                .numpy()
            )
            o = np.squeeze(o).flatten()
            print(np.sum(~np.isnan(o)) / len(o) * 100)
            obs_x = x.flatten()[~np.isnan(o)]
            obs_y = y.flatten()[~np.isnan(o)]
            ax.scatter(obs_x, obs_y, marker=".", s=0.005, c="k")

        ax.xaxis.set_ticklabels([])
        ax.yaxis.set_ticklabels([])
        ax.axes.xaxis.set_visible(False)
        ax.axes.yaxis.set_visible(False)

    plt.suptitle(f"{ttl_header}Time = {np.round(t,2)}")
    plt.tight_layout()

    # if write_out:
    #     fig.savefig(f"{FIG_DIR}/{fig_file_name}.jpg")

    plt.show()

In [ ]:
uhr_omegas, hr_omegas = get_uhr_and_hr_omegas()

In [ ]:
output_obs_npz_file_path = (
    f"{TMP_DATA_DIR}/UHR_seed_{I_SEED_UHR:05}_og{GRID_INTERVAL:02}_{CONFIG_NAME}.npz"
)
all_data = np.load(output_obs_npz_file_path)
hr_obs = torch.from_numpy(all_data["hr_obs"])
sr_forecast = torch.from_numpy(all_data["sr_forecast"])
hr_obs.shape, sr_forecast.shape

In [ ]:
output_lr_file_path = f"{TMP_DATA_DIR}/UHR_seed_{I_SEED_UHR:05}_og{GRID_INTERVAL:02}_SRDA_seed_{OBS_SRDA_SEED}_ens_bicubic_lr.npy"
output_hr_file_path = f"{TMP_DATA_DIR}/UHR_seed_{I_SEED_UHR:05}_og{GRID_INTERVAL:02}_SRDA_seed_{OBS_SRDA_SEED}_ens_bicubic_mean_hr.pickle"
enkf_bicubic_lrs = np.load(output_lr_file_path)
dict_enkf_bicubic_hr_analysis = read_pickle(output_hr_file_path)

enkf_bicubic_lrs.shape

In [ ]:
output_hr_enkf_file_path = f"{TMP_DATA_DIR}/UHR_seed_{I_SEED_UHR:05}_og{GRID_INTERVAL:02}_SRDA_seed_{OBS_SRDA_SEED}_enkf_hr_mean.npy"
enkf_hrs = np.load(output_hr_enkf_file_path)
enkf_hrs.shape

In [ ]:
for i in range(4, NUM_TIMES, 16):
    break
    enkf_bicubic = dict_enkf_bicubic_hr_analysis[i]

    dict_data = {
        "GT (UHR)": uhr_omegas[i].numpy(),
        # "GT (HR)": hr_omegas[i].numpy(),
        "EnKF (Bicubic)": enkf_bicubic,
        "EnKF (HR)": enkf_hrs[i],
        "SRDA (HR)": sr_forecast[i].numpy(),
    }

    t = i * LR_DT * LR_NT

    plot(
        dict_data=dict_data,
        t=t,
        obs=hr_obs[i].numpy(),
        figsize=[16, 4],
        gt_label="GT (UHR)",
    )

In [ ]:
uhr_omegas.shape, sr_forecast.shape

In [ ]:
enkf_mean_lrs = torch.from_numpy(np.mean(enkf_bicubic_lrs, axis=0))
enkf_mean_hrs = torch.from_numpy(enkf_hrs)
enkf_mean_lrs.shape, enkf_mean_hrs.shape

In [ ]:
ssim_func_gauss = SSIM(size_average=False, use_gauss=True)

In [ ]:
df_results = pd.DataFrame()
df_results["Time"] = np.arange(NUM_TIMES) * DT

for res in ["LR", "HR", "UHR"]:
    target_shape = None

    if res == "LR":
        target_shape = (LR_NX, LR_NY)
    if res == "HR":
        target_shape = (HR_NX, HR_NY)
    if res == "UHR":
        target_shape = (UHR_NX, UHR_NY)

    gt = uhr_omegas
    if gt.shape[-2:] != target_shape:
        gt = interpolate(gt, target_shape[0], target_shape[1], "bicubic")

    for label, data in zip(
        ["EnKF", "EnKF(HR)", "SRDA"], [enkf_mean_lrs, enkf_mean_hrs, sr_forecast]
    ):
        if data.shape[-2:] != target_shape:
            data = interpolate(data, target_shape[0], target_shape[1], "bicubic")

        assert data.shape == gt.shape

        mae = torch.mean(torch.abs(data - gt), dim=(1, 2))
        nrm = torch.mean(torch.abs(gt), dim=(1, 2))
        maer = (mae / nrm).numpy()
        mae = mae.numpy()

        assert maer.shape == mae.shape == (NUM_TIMES,)
        df_results[f"{res}_MAER_{label}"] = maer
        df_results[f"{res}_MAE_{label}"] = mae

        ssim = ssim_func_gauss(
            img1=gt[:, None].to(DEVICE),
            img2=data[:, None].to(DEVICE),
        )
        ssim = torch.mean(ssim, dim=(-2, -1)).squeeze().cpu().numpy()
        assert ssim.shape == (NUM_TIMES,)

        df_results[f"{res}_SSIMLoss_{label}"] = 1 - ssim

df_results.to_csv(
    f"{CSV_DATA_DIR}/error_time_series_UHR_seed_{I_SEED_UHR:05}_og{GRID_INTERVAL:02}_SRDA_seed_{OBS_SRDA_SEED}.csv",
    index=False,
)